In [14]:
import pandas as pd
import sqlite3

In [15]:
excel_file = './data/Data-startupticker.xlsx'  
sheet1 = 'Companies'  
sheet2 = 'Deals'

# Read Excel sheets
df1 = pd.read_excel(excel_file, sheet_name=sheet1)
df2 = pd.read_excel(excel_file, sheet_name=sheet2)

# Df preprocessing
df1 = df1.drop_duplicates(subset="Title", keep="first")

# Connect to DB
conn = sqlite3.connect('my_database.db')
conn.execute("PRAGMA foreign_keys = ON;")
cursor = conn.cursor()

# Drop tables if they exist
cursor.execute("DROP TABLE IF EXISTS table2")
cursor.execute("DROP TABLE IF EXISTS table1")
cursor.execute("DROP TABLE IF EXISTS deal")
cursor.execute("DROP TABLE IF EXISTS company")


# Create table1
cursor.execute('''
    CREATE TABLE company (
        Code TEXT,
        Title TEXT PRIMARY KEY,
        Industry TEXT,
        Vertical TEXT,
        Canton TEXT,
        "Spin-offs" TEXT,
        City TEXT,
        Year INTEGER,
        Highlights TEXT,
        "Gender CEO" TEXT,
        OOB BOOLEAN,
        Funded BOOLEAN,
        Comment TEXT
    )
''')

# Create table2 with foreign key to table1.Title
cursor.execute('''
    CREATE TABLE deal (
        Id TEXT PRIMARY KEY,
        Investors TEXT,
        Amount REAL,
        Valuation REAL,
        Comment TEXT,
        URL TEXT,
        Confidential BOOLEAN,
        "Amount confidential" BOOLEAN,
        "Date of the funding round" TEXT,
        Type TEXT,
        Phase TEXT,
        Canton TEXT,
        Company TEXT REFERENCES company(Title),
        "Gender CEO" TEXT
    )
''')

# Insert data from DataFrames
df1.to_sql('company', conn, if_exists='append', index=False)
df2.to_sql('deal', conn, if_exists='append', index=False)


3902

In [16]:
# QUERY TESTING
#print(pd.read_sql("SELECT * FROM table1", conn).head(1))
#print(pd.read_sql("SELECT * FROM table2", conn).head(1))
print( pd.read_sql("SELECT Industry, COUNT(DISTINCT Title) AS number_of_companies FROM company GROUP BY Industry;", conn)) # VALID QUERY
#print( pd.read_sql("SELECT industry, COUNT(DISTINCT company) AS number_of_companies FROM company GROUP BY industry;", conn)) # TXT2SQL QUERY

conn.close()

             Industry  number_of_companies
0                None                 1858
1           Deep Tech                   17
2                 ICT                 1344
3       ICT (fintech)                  353
4              Impact                    6
5   Interdisciplinary                   23
6       Life-Sciences                    2
7             biotech                  311
8           cleantech                  335
9   consumer products                  262
10      healthcare IT                  191
11            medtech                  273
12       micro / nano                  235
